## Reformulations in Mixed-Integer Linear Optimization
So far, we've been looking at _mixed-Integer linear optimization_, which is linear optimization with a new primitive: integrality conditions on some of the variables.

$$
\begin{align*}
    \min_{x}\quad& c^Tx \\
    \text{s.t.}\quad& Ax \leq b \\
    & x \in \mathbb{R}^{m} \times \mathbb{Z}^n
\end{align*}
$$

But in practice, there might be **nonlinearities** in the problem we are trying to model. In our earlier session on callback functions, we've seen one way to handle them for convex functions through a polyhedral outer approximation. In this session, we look at techniques for handling other forms of nonlinearities.

## Logical Constraints (Boolean logic)

There is a natural mapping to Boolean logics `TRUE` and `FALSE` to the binary variables `1` and `0`.  It is also quite common to want to express logical constraints and implications, e.g. "if this factory is built, we can build no other factories in this year", or "either factory A or factory B can be built, but not both". The "trick" is expressing these logical constraints and implications as **linear inequalities**, and sometimes we need to introduce **auxiliary variables**.

We'll start with modeling simple logical statements - `AND` & `OR`.

### AND

"`z` is TRUE if and only if `x` and `y` are both TRUE" (``z = x AND y``)

* `0 AND 0 = 0`
* `1 AND 0 = 0`
* `0 AND 1 = 0`
* `1 AND 1 = 1`

We can write this as
$$z \geq x + y - 1  ,\quad  z \leq x  ,\quad  z \leq y,\quad 0 \leq z \leq 1$$
which we can check by putting the values in (blackboard).

### OR

"`z` is TRUE if and only if either `x` or `y` or both are TRUE" (``z = x OR y``)

* `0 OR 0 = 0`
* `1 OR 0 = 1`
* `0 OR 1 = 1`
* `1 OR 1 = 1`

This can also be expressed by linear inequalities, with no auxiliaries required:
$$z \geq x  ,\quad  z \geq y  ,\quad  z \leq x + y,\quad 0 \leq z \leq 1$$


### XOR

This one is a bit trickier: "`z` is TRUE if and only if either `x` or `y` is TRUE, but not both" (``z = x XOR y``)

* `0 XOR 0 = 0`
* `1 XOR 0 = 1`
* `0 XOR 1 = 1`
* `1 XOR 1 = 0`

One way to do it is `z = x XOR y = (x OR y) - (x AND y)`, but then would need two auxiliary variables, e.g. `z1 = x OR y`, `z2 = x AND y`, and then finally `z = z1 - z2`. Now its your turn:

> **[Exercise]**: Logical Constraints

> Express `z = x XOR y` with using ONE auxiliary variable. Can you do it with NO auxiliary variables?

**Remark**: For more, this [quick reference guide (pdf)](http://brblog.typepad.com/files/mipformref-1.pdf) presents a collection of MIP model formulations, including standard linearization techniques involving binary variables, the use of more specific modeling objects such as SOS and partial integer variables, and reformulations of logic constraints through indicator constraints.

## Piecewise constant functions

> Lots more to learn about this in [Juan Pablo's review](http://www.mit.edu/~jvielma/publications/index.html#Mixed-Integer-Linear-Programming-Formulation-Techniques) of formulation techniques!

In this section we are going to try to model the following type of relationship:

$$
z(x) = \begin{cases}
a_{1} & \quad r_{1}\leq x\leq r_{2}\\
a_{2} & \quad r_{2}\leq x\leq r_{3}\\
\vdots\\
a_{n-1} & \quad r_{n-1}\leq x\leq r_{n}
\end{cases}
$$

where $z$ is a piecewise constant function of the variable $x$.

1) To do so, we first create auxiliary binary variables $y_1,\dots,y_n$, for which we wish to express the following relationship:

$$ x \geq r_i \Longrightarrow y_i = 1 $$

> which we accomplish with the following linear inequalities

> \begin{alignat*}{2}
    y_i &\geq   &(x - r_{i})/M \\
    y_i &\leq 1-&(x - r_{i})/M
\end{alignat*}

2) We then define

$$ \delta_i = a_i - a_{i-1} $$

where $a_{0}=0$.

3) Finally, we express $z$ in the following way:

$$ z = \sum_{i=1}^{n-1} \delta_i y_i $$

## Unions of polyhedra

The piecewise linear modeling above can be seen as a special case of a more general modeling paradigm:

$$
\begin{align*}
\min_{x}\quad& c^Tx \\
\text{s.t.}\quad& x \in \bigcup_{i=1}^m P_i
\end{align*}
$$

where $P_i = \{x : A^i x \leq b^i\}$ are polyhedra. We'll assume for today that $P_1,\dots,P_m$ are bounded (i.e. polytopes); for a more general treatment, take 15.083J.

> **[Exercise]**: Unions of polyhedra

> How do we formulate this?

## Discussion
When else might you expect to apply similar techniques? Some examples:

- [Stochastic Network Interdiction (pdf)](http://homepages.cae.wisc.edu/~linderot/papers/Janjarassuk-Linderoth-06-TR.pdf) -- discussion of linearization on page 5, and resulting formulation on page 6
- [Optimal Classification Trees (pdf)](http://jack.dunn.nz/papers/OptimalClassificationTrees.pdf) -- discussion of linearization from pages 10-12, and resulting formulation on page 12

## Discussion
Is mixed integer linear optimization "special"? No. The solvers might be less mature, but if you're interested in the class of conic and semi-definite optimization problems, there are more techniques that will be available to you. We refer the interested reader to the [MOSEK modeling manual](http://docs.mosek.com/generic/modeling-letter.pdf) for some pointers.